## Webscrapping from metroscubicos.com

In [1]:
# import instances and libraries
from splinter import Browser
from bs4 import BeautifulSoup as soup
from webdriver_manager.chrome import ChromeDriverManager
import requests
import re
import pandas as pd
# To show a progess bar in loops:
from tqdm import tqdm_notebook as tqdm
import csv

In [2]:
# List all entities available in metroscubicos.com

entidades = ['Aguascalientes', 'Baja California', 'Baja California Sur', 'Campeche', 'Chiapas', 'Chihuahua', 'Coahuila', 'Colima',
            'Distrito Federal', 'Durango', 'Estado De México', 'Guanajuato', 'Guerrero', 'Hidalgo', 'Jalisco', 'Michoacán',
             'Morelos', 'Nayarit', 'Nuevo León', 'Oaxaca', 'Puebla', 'Querétaro', 'Quintana Roo', 'San Luis Potosí',
             'Sinaloa', 'Sonora', 'Tabasco', 'Tamaulipas', 'Tlaxcala', 'Veracruz', 'Yucatán', 'Zacatecas']

In [3]:
# make smallcaps and change spaces for dashes, and select which entities
entidadesLower = [x.lower().replace(' ', '-') for x in entidades[3:]]

In [4]:
entidadesLower

['campeche',
 'chiapas',
 'chihuahua',
 'coahuila',
 'colima',
 'distrito-federal',
 'durango',
 'estado-de-méxico',
 'guanajuato',
 'guerrero',
 'hidalgo',
 'jalisco',
 'michoacán',
 'morelos',
 'nayarit',
 'nuevo-león',
 'oaxaca',
 'puebla',
 'querétaro',
 'quintana-roo',
 'san-luis-potosí',
 'sinaloa',
 'sonora',
 'tabasco',
 'tamaulipas',
 'tlaxcala',
 'veracruz',
 'yucatán',
 'zacatecas']

In [5]:
# Initialize variables
cityList = []
listLinks = []
tipos = ['casas', 'departamentos', 'terrenos']
baseUrl = 'https://inmuebles.metroscubicos.com/'
csvPath = './Resources/'

### Script para hacer el webscrapping por ciudad

In [ ]:
## Read file by city
ciudad = entidadesLower[0]
csvfile = csvPath+ciudad+".csv"
data= pd.read_csv(csvfile)
data.head()

In [ ]:
# Select which city
ciudades = entidadesLower[0:2]
ciudades

In [ ]:
for ciudad in tqdm(entidadesLower):
    # Read csv file and get links
    csvfile = csvPath+ciudad+".csv"
    data= pd.read_csv(csvfile)
    links = data["link"].tolist()
    
    # Loop through all links and webscrape info. Info is stores in a dictionary
    for link in tqdm(links):
        # Clean dictionary for each link
        dict = {}
        dict["link"] = link
        try: 
            asset_soup = soup(requests.get(link).content, 'html.parser')
        except:
            print(f"Cannot retrieve {link} ... passing")
            continue
        # Get price and currency symbol
        if asset_soup.find('span', class_="andes-money-amount__currency-symbol") != None:
            simbolo = asset_soup.find('span', class_="andes-money-amount__currency-symbol").get_text()
        else:
            simbolo = ""
        if asset_soup.find('span', class_="andes-money-amount__fraction") != None:
            monto = asset_soup.find('span', class_="andes-money-amount__fraction").get_text()
        else:
            monto = ""
        precio = simbolo+monto
        dict["precio"] = precio
        if asset_soup.find('ol', class_="andes-breadcrumb") != None:
            divBreadcrumbs = asset_soup.find('ol', class_="andes-breadcrumb")
            aBreadcrumbs = divBreadcrumbs.find_all('a')
            if len(aBreadcrumbs) > 1:
                dict['municipio'] = aBreadcrumbs[-2].get_text()
                dict['colonia'] = aBreadcrumbs[-1].get_text()
            elif len(aBreadcrumbs) == 1:
                dict['municipio'] = ""
                dict['colonia'] = aBreadcrumbs[-1].get_text()
            else:
                dict['municipio'] = ""
                dict['colonia'] = ""
        # Find table and process columns and rows
        if asset_soup.find('tbody', class_="andes-table__body") == None:
            tabla = ""
            pass
        else:
            tabla = asset_soup.find('tbody', class_="andes-table__body")
            tablaDatos = asset_soup.find('table', class_="andes-table") 
            nombreDatos = tablaDatos.find_all('th')
            dataDatos = tablaDatos.find_all('td')
            for i in range(len(nombreDatos)):
                dict[nombreDatos[i].get_text()] = dataDatos[i].get_text()
        
        # Add dictionary values to global list
        if dict not in cityList:
            cityList.append(dict)

    results_df = pd.DataFrame(cityList)
    final_df = pd.merge(data, results_df, on="link")
    file = csvPath+ciudad+"-webscrapeResults.csv"
    final_df.to_csv(file, index=False)
            

C:\Users\CAC\anaconda3\envs\PythonData\lib\site-packages\ipykernel_launcher.py:1: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  """Entry point for launching an IPython kernel.


  0%|          | 0/29 [00:00<?, ?it/s]

C:\Users\CAC\anaconda3\envs\PythonData\lib\site-packages\ipykernel_launcher.py:8: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  


  0%|          | 0/440 [00:00<?, ?it/s]

  0%|          | 0/2998 [00:00<?, ?it/s]

Cannot retrieve https://casa.metroscubicos.com/MLM-1399783437-vn-hermosa-casa-en-venta-ubicada-en-tapachula-chiapas-_JM ... passing
Cannot retrieve https://casa.metroscubicos.com/MLM-1399913257-vn-hermosa-casa-en-venta-ubicada-en-tapachula-chiapas-_JM ... passing
Cannot retrieve https://casa.metroscubicos.com/MLM-1388768819-casa-sola-en-venta-paseo-del-bosque-_JM ... passing
Cannot retrieve https://casa.metroscubicos.com/MLM-1365304031-casa-sola-en-venta-estrella-de-oriente-_JM ... passing
Cannot retrieve https://casa.metroscubicos.com/MLM-952438252-casa-en-venta-fracc-san-fernando-_JM ... passing
Cannot retrieve https://casa.metroscubicos.com/MLM-936377365-casa-sola-en-venta-las-granjas-_JM ... passing
Cannot retrieve https://casa.metroscubicos.com/MLM-1406071681-sc-cv-420-copoya-casa-en-venta-_JM ... passing
Cannot retrieve https://casa.metroscubicos.com/MLM-1403519315-casa-sola-en-venta-san-jose-teran-_JM ... passing
Cannot retrieve https://casa.metroscubicos.com/MLM-1399770451-vn-h

Cannot retrieve https://casa.metroscubicos.com/MLM-1410737155-sc-cv-510-castillo-tielmans-casa-en-venta-_JM ... passing
Cannot retrieve https://casa.metroscubicos.com/MLM-1392405828-casa-sola-en-venta-campanario-_JM ... passing
Cannot retrieve https://casa.metroscubicos.com/MLM-1384424162-casa-sola-en-venta-colinas-del-sur-_JM ... passing
Cannot retrieve https://casa.metroscubicos.com/MLM-1384398251-casa-sola-en-venta-barrio-colon-_JM ... passing
Cannot retrieve https://casa.metroscubicos.com/MLM-1384398252-casa-sola-en-venta-barrio-santo-tomas-_JM ... passing
Cannot retrieve https://casa.metroscubicos.com/MLM-1353240146-sccv455-se-vende-casa-en-plan-ayala-_JM ... passing
Cannot retrieve https://casa.metroscubicos.com/MLM-1390451533-casa-sola-en-venta-el-retiro-_JM ... passing
Cannot retrieve https://casa.metroscubicos.com/MLM-1384385402-casa-sola-en-venta-teran-_JM ... passing
Cannot retrieve https://casa.metroscubicos.com/MLM-1353723692-sccv457-se-vende-casa-en-san-jose-teran-_JM ...

  0%|          | 0/3613 [00:00<?, ?it/s]

### Multithreading

In [ ]:
from multiprocessing.dummy import Pool  # This is a thread-based Pool
from multiprocessing import cpu_count

In [ ]:
def parseWeb(url):
    dict = {}
    try: 
        asset_soup = soup(requests.get(url).content, 'html.parser')
    except:
        print(f"Cannot retrieve {url} ... passing")
        return dict
    # Get price and currency symbol
    if asset_soup.find('span', class_="andes-money-amount__currency-symbol") != None:
        simbolo = asset_soup.find('span', class_="andes-money-amount__currency-symbol").get_text()
    else:
        simbolo = ""
    if asset_soup.find('span', class_="andes-money-amount__fraction") != None:
        monto = asset_soup.find('span', class_="andes-money-amount__fraction").get_text()
    else:
        monto = ""
    precio = simbolo+monto
    dict["precio"] = precio
    if asset_soup.find('ol', class_="andes-breadcrumb") != None:
        divBreadcrumbs = asset_soup.find('ol', class_="andes-breadcrumb")
        aBreadcrumbs = divBreadcrumbs.find_all('a')
        dict['colonia'] = aBreadcrumbs[-1].get_text()
        dict['municipio'] = aBreadcrumbs[-2].get_text()
    # Find table and process columns and rows
    if asset_soup.find('tbody', class_="andes-table__body") == None:
        tabla = ""
        pass
    else:
        tabla = asset_soup.find('tbody', class_="andes-table__body")
        tablaDatos = asset_soup.find('table', class_="andes-table") 
        nombreDatos = tablaDatos.find_all('th')
        dataDatos = tablaDatos.find_all('td')
        for i in range(len(nombreDatos)):
            dict[nombreDatos[i].get_text()] = dataDatos[i].get_text()
    #if dict not in cityList:
    #    cityList.append(dict)
    return dict

In [ ]:
if __name__ == '__main__':
    ciudad = entidadesLower[0]
    csvfile = csvPath+ciudad+".csv"
    data= pd.read_csv(csvfile)
    links = data["link"].tolist()
    pool = Pool(cpu_count() * 200)
    results = pool.map(parseWeb, links)
    if results not in cityList and len(results) != 0:
        cityList.append(results)
        print(f"Processed {len(results)} links")